In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)


In [7]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        print(covar_x.shape)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [8]:

# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    #print(output.covariance_matrix.shape)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

torch.Size([100, 100])
Iter 1/50 - Loss: 0.939   lengthscale: 0.693   noise: 0.693
torch.Size([100, 100])
Iter 2/50 - Loss: 0.908   lengthscale: 0.644   noise: 0.644
torch.Size([100, 100])
Iter 3/50 - Loss: 0.875   lengthscale: 0.598   noise: 0.598
torch.Size([100, 100])
Iter 4/50 - Loss: 0.837   lengthscale: 0.555   noise: 0.554
torch.Size([100, 100])
Iter 5/50 - Loss: 0.795   lengthscale: 0.514   noise: 0.513
torch.Size([100, 100])
Iter 6/50 - Loss: 0.748   lengthscale: 0.475   noise: 0.474
torch.Size([100, 100])
Iter 7/50 - Loss: 0.697   lengthscale: 0.439   noise: 0.437
torch.Size([100, 100])
Iter 8/50 - Loss: 0.646   lengthscale: 0.405   noise: 0.402
torch.Size([100, 100])
Iter 9/50 - Loss: 0.597   lengthscale: 0.372   noise: 0.369
torch.Size([100, 100])
Iter 10/50 - Loss: 0.553   lengthscale: 0.342   noise: 0.339
torch.Size([100, 100])
Iter 11/50 - Loss: 0.513   lengthscale: 0.315   noise: 0.310
torch.Size([100, 100])
Iter 12/50 - Loss: 0.476   lengthscale: 0.292   noise: 0.284
t